# 1: Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import pickle
import os

# Librerías de Scikit-Learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Verificar que la carpeta de modelos existe, si no, crearla
if not os.path.exists('../models'):
    os.makedirs('../models')
    print("Carpeta '../models' creada.")

# 2: Carga y Limpieza de Datos

In [ ]:
# Carga y Limpieza de Datos
try:
    df = pd.read_csv('../dataset/penguins_size.csv')
    print("Dataset cargado correctamente.")
except FileNotFoundError:
    print("Error: No se encuentra el archivo. Verifica que esté en 'proyecto_penguins/data/'")

# Eliminar filas con valores nulos (NA)
initial_count = len(df)
df = df.dropna()
final_count = len(df)

print(f"Filas originales: {initial_count}, Filas tras limpieza: {final_count}")
df.head()

Dataset cargado correctamente.
Filas originales: 344, Filas tras limpieza: 334


,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE


# 3: Definición de Variables y Mapeo

In [3]:
# Definir variables
target = 'species'
numerical = ['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']
categorical = ['island', 'sex']

# Convertir target a numérico (Recomendado para algunos modelos de sklearn)
species_map = {'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}
df['species_num'] = df['species'].map(species_map)

print("Mapeo de especies:", species_map)
df[['species', 'species_num']].head()

Mapeo de especies: {'Adelie': 0, 'Chinstrap': 1, 'Gentoo': 2}


,species,species_num
0,Adelie,0
1,Adelie,0
2,Adelie,0
4,Adelie,0
5,Adelie,0


# 4: División del Dataset (Train / Test Split)

In [4]:
# Split 80/20
# Separamos el 80% para entrenar y el 20% para testear
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Extraemos la variable objetivo (y)
y_train = df_train['species_num'].values
y_test = df_test['species_num'].values

print(f"Datos de entrenamiento: {len(df_train)}")
print(f"Datos de prueba: {len(df_test)}")

Datos de entrenamiento: 267
Datos de prueba: 67


# 5: Preprocesamiento - Escalado Estándar (Variables Numéricas)

In [5]:
# Preprocesamiento: Escalado de Numéricas
# Es importante escalar para que modelos como SVM o KNN funcionen bien
scaler = StandardScaler()

# Ajustamos (fit) SOLO con los datos de entrenamiento para evitar data leakage
scaler.fit(df_train[numerical])

# Transformamos ambos conjuntos
# Nota: Esto modifica los dataframes temporalmente para generar los dicts después
df_train[numerical] = scaler.transform(df_train[numerical])
df_test[numerical] = scaler.transform(df_test[numerical])

print("Variables numéricas escaladas (Media ~0, Desviación ~1)")
df_train[numerical].head()

Variables numéricas escaladas (Media ~0, Desviación ~1)


,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
230,-0.596447,-1.743053,0.921649,0.533572
84,-1.265806,0.327614,-0.724133,-1.073743
303,1.095541,-0.631963,1.637206,1.399049
22,-1.526111,1.034672,-0.867245,-0.517365
29,-0.670821,0.883159,-1.511246,-0.331905


# 6: Preprocesamiento - One-Hot Encoding (Variables Categóricas)

In [6]:
# Preprocesamiento: DictVectorizer (One-Hot Encoding)
# Convertimos el dataframe a lista de diccionarios
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
test_dicts = df_test[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)

# Entrenamos el vectorizador con los datos de train y transformamos
X_train = dv.fit_transform(train_dicts)
X_test = dv.transform(test_dicts)

print(f"Forma de la matriz de entrenamiento: {X_train.shape}")
print("Features generadas:", dv.get_feature_names_out())

Forma de la matriz de entrenamiento: (267, 10)
Features generadas: ['body_mass_g' 'culmen_depth_mm' 'culmen_length_mm' 'flipper_length_mm'
 'island=Biscoe' 'island=Dream' 'island=Torgersen' 'sex=.' 'sex=FEMALE'
 'sex=MALE']


# 7: Definición, Entrenamiento y Evaluación de Modelos

In [7]:
# Definir y Entrenar los 4 Modelos
models = {
    'logistic_regression': LogisticRegression(solver='liblinear'),
    'svm': SVC(kernel='linear', probability=True), # probability=True es vital para luego usar predict_proba
    'decision_tree': DecisionTreeClassifier(max_depth=5),
    'knn': KNeighborsClassifier(n_neighbors=5)
}

results = {}

print("--- Iniciando Entrenamiento ---")
for name, model in models.items():
    # Entrenar
    model.fit(X_train, y_train)
    
    # Evaluar
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    
    print(f"Modelo: {name:20} | Accuracy: {acc:.4f}")

    # 7. Serializar (Guardar Scaler, DV y Modelo juntos)
    # Guardamos los 3 objetos necesarios para predecir en el futuro
    with open(f'../models/{name}.pck', 'wb') as f:
        pickle.dump((scaler, dv, model), f)
    print(f"   -> Guardado en ../models/{name}.pck")

--- Iniciando Entrenamiento ---
Modelo: logistic_regression  | Accuracy: 1.0000
   -> Guardado en ../models/logistic_regression.pck
Modelo: svm                  | Accuracy: 1.0000
   -> Guardado en ../models/svm.pck
Modelo: decision_tree        | Accuracy: 1.0000
   -> Guardado en ../models/decision_tree.pck
Modelo: knn                  | Accuracy: 0.9851
   -> Guardado en ../models/knn.pck


c:\Users\leox_\anaconda3\envs\cas-practic\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


# 8: Resumen de Resultados

In [8]:
# Visualizar qué modelo funcionó mejor
results_df = pd.DataFrame(list(results.items()), columns=['Modelo', 'Accuracy'])
results_df.sort_values(by='Accuracy', ascending=False)

,Modelo,Accuracy
0,logistic_regression,1.000000
1,svm,1.000000
2,decision_tree,1.000000
3,knn,0.985075
